Purpose of this notebook is to explore the semantic search use case with browsing history in mind
- Important caveat is to explore the support for multiple languges

Reference link -> https://data.firefox.com/dashboard/usage-behavior

  Worldwide, English (US) remains the most common, at about 40% of the population, with German (11%) and French (8.1%) coming 2nd and 3rd. Simplified Chinese is the 4th most common language (6.7%), and Spanish (Spain) is the 5th most common language (5%).

In [ ]:
import pandas as pd
import onnxruntime as ort
from transformers import AutoTokenizer
import numpy as np
import requests
import os
import sys

In [ ]:
# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

In [ ]:
from src.constants import EMBEDDING_MODELS_DICT
from src.feature_extractor import FeatureExtractor

In [ ]:
# !cp /tmp/output_file.txt /Users/cgopal/Downloads/places_output_file_v1.txt

#### Lets try reading browsing history

Download browsing history:

1) cp "/Users/<username>/Library/Application Support/Firefox/Profiles/<profilename>/places.sqlite" /tmp/places.sqlite
2) sqlite3 /tmp/places.sqlite
3) within sqlite run below commands one by one
```
.schema moz_places
.output output_file.txt
SELECT url, title, description, preview_image_url, frecency, last_visit_date FROM moz_places WHERE description  NOTNULL ORDER BY last_visit_date DESC limit 600
```
4) copy the file output_file to ~/Downloads/places_output_file_v1.txt


In [ ]:
history = pd.read_csv("/Users/cgopal/Downloads/places_output_file_v1.txt",
                      sep="|", engine="python", header=None, encoding="utf-8", on_bad_lines="skip", index_col=False,
                      names=['url', 'title', 'description', 'preview_image_url', 'frecency', 'last_visit_date'])

print(len(history))
history.head().T

In [ ]:
history['last_visit_date'] = pd.to_datetime(history['last_visit_date'], unit='us')
history['combined_text'] = history['title'].fillna('') + " " + history['description'].fillna('')
history = history.loc[history['combined_text'] != ''].reset_index(drop=True)

print(len(history))

In [ ]:
history

In [ ]:
EMBEDDING_MODELS_DICT

In [ ]:
texts = history['combined_text'].values.tolist()
embeddings_dict = {}
embeddings_sizes = {}

for model in EMBEDDING_MODELS_DICT.keys():
    fe = FeatureExtractor(EMBEDDING_MODELS_DICT, model_name=model)
    embeddings_dict[model] = fe.get_embeddings(texts)
    print(model, embeddings_dict[model].shape)
    embeddings_sizes[model] = embeddings_dict[model].shape[1]


In [ ]:
embeddings_sizes

In [ ]:
embeddings_dict.keys()

In [ ]:
embeddings_dict['Xenova/paraphrase-multilingual-mpnet-base-v2'].shape

In [ ]:
!mkdir -p ../data

In [ ]:
import pickle

with open("../data/embeddings_dict.pkl", "wb") as f:
    pickle.dump(embeddings_dict, f)

with open("../data/embeddings_sizes.pkl", "wb") as f:
    pickle.dump(embeddings_sizes, f)

history.to_csv("../data/history.csv", index=False)

#### Explore sqlite vector DB

In [ ]:
import numpy as np
import sqlite3
import sqlite_vec

from typing import List
import struct

In [ ]:

def serialize_f32(vector: List[float]) -> bytes:
    """serializes a list of floats into a compact "raw bytes" format"""
    return struct.pack("%sf" % len(vector), *vector)

In [ ]:
db = sqlite3.connect(":memory:")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

sqlite_version, vec_version = db.execute(
    "select sqlite_version(), vec_version()"
).fetchone()
print(f"sqlite_version={sqlite_version}, vec_version={vec_version}")

In [ ]:
path = "../data/embeddings_dict.pkl"

with open(path, "rb") as f:
    embeddings_dict = pickle.load(f)

In [ ]:
embeddings_dict.keys()

In [ ]:
# model_name = "Xenova/paraphrase-multilingual-MiniLM-L12-v2"
model_name = "Xenova/distiluse-base-multilingual-cased-v1"
# model_name = "Xenova/all-MiniLM-L6-v2"
# model_name = "nomic-ai/nomic-embed-text-v1.5"
EMBEDDING_SIZE = embeddings_sizes[model_name]

In [ ]:
items = []
for idx, vec in enumerate(embeddings_dict[model_name]):
    items.append((idx, list(vec)))

In [ ]:
model_name_normalized = model_name.replace("/","_").replace("-","_").replace(".","_")

In [ ]:
db.execute(f"CREATE VIRTUAL TABLE vec_items_{model_name_normalized} USING vec0(embedding float[{EMBEDDING_SIZE}])")

with db:
    for item in items:
        db.execute(
            f"INSERT INTO vec_items_{model_name_normalized}(rowid, embedding) VALUES (?, ?)",
            [item[0], serialize_f32(item[1])],
        )



In [ ]:
history = pd.read_csv("../data/history.csv")

In [ ]:
query = "github"

fe = FeatureExtractor(EMBEDDING_MODELS_DICT, model_name=model_name)
query_embedding = fe.get_embeddings([query])[0]


In [ ]:
query_embedding.shape

In [ ]:
rows = db.execute(
    f"""
      SELECT
        rowid,
        distance
      FROM vec_items_{model_name_normalized}
      WHERE embedding MATCH ?
      ORDER BY distance
      LIMIT 3
    """,
    [serialize_f32(query_embedding)],
).fetchall()

print(rows)

In [ ]:
pd.set_option('display.max_colwidth', 200)

In [ ]:
print(f"query = {query}")
# history.iloc[[row for row, score in rows]]
row_indices = [row for row, score in rows]
distance = [score for row, score in rows]

selected_rows = history.iloc[row_indices].copy()
selected_rows["distance"] = distance
selected_rows